In [2]:
!pip install ultralytics
!pip install shapely

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.1/872.1 kB 18.0 MB/s eta 0:00:0000:01


In [3]:
from ultralytics import YOLO
from shapely.geometry import Point, Polygon
import cv2

def getVehicleCount(path,incoming_lane_polygon):

    cap = cv2.VideoCapture(path)
    ret, frame = cap.read()
    if not ret:
        raise RuntimeError("Failed to read video")

    frame_height, frame_width = frame.shape[:2]

    model = YOLO('/kaggle/input/custom_model/other/yolo_model/1/best.pt')
    
    # Scale the polygon coordinates to the actual frame size
    scaled_polygon = [(int(x * frame_width), int(y * frame_height)) for x, y in incoming_lane_polygon]

    def is_point_in_polygon(point, polygon):
        point = Point(point)
        polygon = Polygon(polygon)
        return polygon.contains(point)

    results = model.track(source=path, save=True, tracker="bytetrack.yaml")

    unique_vehicle_ids = set()
    vehicle_ids = set()
    lanes = []

    emergency_vehicle_present = False

    for frame_idx, result in enumerate(results):
        if result.boxes is not None:
            for box in result.boxes:
                if box.id is not None:
                    # Extract the bounding box coordinates
                    xmin, ymin, xmax, ymax = box.xyxy[0]

                    # Get the center point of the bounding box
                    x_center = (xmin + xmax) / 2
                    y_center = (ymin + ymax) / 2
                    center_point = (x_center, y_center)
                    vehicle_ids.add(int(box.id))

                    # Check if the center point is inside the incoming lane polygon
                    if is_point_in_polygon(center_point, scaled_polygon):
                        unique_vehicle_ids.add(int(box.id))
                        # Check the class ID for emergency vehicle
                        if box.cls == 0:
                            emergency_vehicle_present = True

    # Append True or False based on the presence of an emergency vehicle
    lanes.append(emergency_vehicle_present)

    # Append the vehicle count to the list
    lanes.append(len(unique_vehicle_ids))

    return lanes


In [4]:
#for single lane vehicles

from ultralytics import YOLO
from shapely.geometry import Point, Polygon
import cv2

def getVehicleCount2(path):

    cap = cv2.VideoCapture(path)
    ret, frame = cap.read()
    if not ret:
        raise RuntimeError("Failed to read video")

    frame_height, frame_width = frame.shape[:2]

    model = YOLO('/kaggle/input/custom_model/other/yolo_model/1/best.pt')

    results = model.track(source=path, save=True, tracker="bytetrack.yaml")

    vehicle_ids = set()
    lanes = []

    emergency_vehicle_present = False

    for frame_idx, result in enumerate(results):
        if result.boxes is not None:
            for box in result.boxes:
                if box.id is not None:
                    # Extract the bounding box coordinates
                    xmin, ymin, xmax, ymax = box.xyxy[0]

                    # Get the center point of the bounding box
                    x_center = (xmin + xmax) / 2
                    y_center = (ymin + ymax) / 2
                    center_point = (x_center, y_center)
                    vehicle_ids.add(int(box.id))

                    
                    # Check the class ID for emergency vehicle
                    if box.cls == 0:
                        emergency_vehicle_present = True

    # Append True or False based on the presence of an emergency vehicle
    lanes.append(emergency_vehicle_present)

    # Append the vehicle count to the list
    lanes.append(len(vehicle_ids))

    return lanes


In [5]:
lane1=getVehicleCount('/kaggle/input/traffic-video/test1.mp4',[(0.59, 0.76), (0.87, 0.72), (0.64, 0.45), (0.53, 0.45)] )
lane2=getVehicleCount2('/kaggle/input/traffic-video/test2.mp4')
lane3=getVehicleCount2('/kaggle/input/traffic-video/test3.mp4')
lane4=getVehicleCount('/kaggle/input/traffic-video/test4.mp4',[(0.04, 0.76),(0.46, 0.77),(0.44, 0.33),(0.3, 0.34)] )

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.4 MB/s eta 0:00:00 0:00:01

requirements: AutoUpdate success ✅ 11.5s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/942) /kaggle/input/traffic-video/test1.mp4: 224x384 1 bus, 3 cars, 1 pickup, 82.3ms
video 1/1 (frame 2/9

In [6]:
print(lane1)
print(lane2)
print(lane3)
print(lane4)

[False, 33]
[False, 40]
[True, 155]
[False, 5]


In [9]:
import heapq

class Lane:
    def __init__(self, id, has_emergency_vehicle,  vehicle_count):
        self.id = id
        self.vehicle_count = vehicle_count
        self.has_emergency_vehicle = has_emergency_vehicle

    def __lt__(self, other):
        if self.has_emergency_vehicle != other.has_emergency_vehicle:
            return self.has_emergency_vehicle > other.has_emergency_vehicle
        return self.vehicle_count > other.vehicle_count

def schedule_lanes(lanes, base_time_per_vehicle=2):
    # Separate emergency and non-emergency lanes
    emergency_lanes = [lane for lane in lanes if lane.has_emergency_vehicle]
    non_emergency_lanes = [lane for lane in lanes if not lane.has_emergency_vehicle]

    # Create a priority queue for non-emergency lanes
    heapq.heapify(non_emergency_lanes)

    schedule = []
    total_time = 0

    # First, process all emergency lanes
    for lane in emergency_lanes:
        time = max(base_time_per_vehicle * lane.vehicle_count, 10)  # Minimum time for emergency
        schedule.append((lane.id, time))
        total_time += time

    # Then, process non-emergency lanes based on vehicle count
    while non_emergency_lanes:
        lane = heapq.heappop(non_emergency_lanes)
        time = base_time_per_vehicle * lane.vehicle_count
        schedule.append((lane.id, time))
        total_time += time

    return schedule, total_time


lanes = [
    Lane(1, lane1[0], lane1[1]),
    Lane(2, lane2[0], lane2[1]),
    Lane(3, lane3[0], lane3[1]),
    Lane(4, lane4[0], lane4[1])
]

schedule, total_time = schedule_lanes(lanes)

print("Lane schedule:")
for lane_id, time in schedule:
    print(f"Lane {lane_id}: {time} seconds")

print(f"Total time for all lanes: {total_time} seconds")

Lane schedule:
Lane 3: 310 seconds
Lane 2: 80 seconds
Lane 1: 66 seconds
Lane 4: 10 seconds
Total time for all lanes: 466 seconds
